In [5]:
import mysql.connector
import os
import datetime




In [ ]:
# Connect to MySQL
# conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="Password@123",
#     database="AttendanceDB"
# )

In [ ]:
# to create a cursor object in MySQL, which allows us to execute SQL queries in Python.
# It is used to execute queries like SELECT, INSERT, UPDATE, and DELETE.

# cursor=conn.cursor()


# # Student details
# students = [
#     ("Anushka", "/home/user/Deep_Learning1/ATTENDANCE/class_list/Anushka.jpeg"),
#     ("jordon", "/home/user/Deep_Learning1/ATTENDANCE/class_list/jordon.jpeg"),
#     ("madhuri", "/home/user/Deep_Learning1/ATTENDANCE/class_list/madhuri.jpeg"),
#     ("sandra", "/home/user/Deep_Learning1/ATTENDANCE/class_list/sandra.jpeg"),
#     ("sidhi", "/home/user/Deep_Learning1/ATTENDANCE/class_list/sidhi.jpeg"),
#     ("Akash","/home/user/Deep_Learning1/ATTENDANCE/class_list/Akash.jpg")  
# ]
# # Insert data
# query = "INSERT INTO STUDENTS_RECORD ( NAME, IMAGE_PATH) VALUES (%s, %s)"
# cursor.executemany(query, students)

# # Commit the transaction (Save changes)
# conn.commit()
# #  Close the connection
# cursor.close()
# conn.close()

# print("Student records inserted successfully!")

Student records inserted successfully!


In [ ]:

# # Connect to MySQL
# conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="Password@123",
#     database="AttendanceDB"
# )

# cursor = conn.cursor()

# # Folder containing student images
# image_folder = "/home/user/Deep_Learning1/ATTENDANCE/class_list"
# print(os.listdir(image_folder))

# # Insert student names and paths automatically
# for filename in os.listdir(image_folder):
#     if filename.endswith(".jpg") or filename.endswith(".png"):
#         name = os.path.splitext(filename)[0]  # Extract name from filename
#         image_path = os.path.join(image_folder, filename)
        
#         query = "INSERT INTO STUDENTS_RECORD ( NAME, IMAGE_PATH) VALUES (%s, %s)"
#         cursor.execute(query, (name, image_path))

# # Commit and close
# conn.commit()
# cursor.close()
# conn.close()

# print("All student images have been inserted into the database!")

['madhuri.jpeg', 'Anushka.jpeg', 'sandra.jpeg', 'Akash.jpg', 'sidhi.jpeg', 'jordon.jpeg']


In [3]:
import cv2
import numpy as np
import face_recognition
import mysql.connector

def fetch_students():
    conn = mysql.connector.connect(host="localhost", user="root", password="Password@123", database="AttendanceDB")
    cursor = conn.cursor()
    cursor.execute("SELECT name, image_path FROM STUDENTS_RECORD")
    students = cursor.fetchall()
    # Gets all rows from the executed query and stores them in the students variable.
    conn.close()
    return students

def encode_faces(students):
    encoded_faces = {}
    for name, img_path in students:
        image = face_recognition.load_image_file(img_path)
        encoding = face_recognition.face_encodings(image)
        if encoding:
            encoded_faces[name] = encoding[0]
    return encoded_faces




In [6]:
def mark_attendance(name):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Password@123",
        database="AttendanceDB"
    )
    cursor = conn.cursor()

    # Check if the student has already been marked present today
    cursor.execute(
        "SELECT * FROM ATTENDANCE_PRESENT WHERE NAME = %s AND DATE = CURDATE()",
        (name,)
    )
    result = cursor.fetchone()  # Fetch one record

    if result is None:  # If no record exists, insert attendance
        cursor.execute(
            "INSERT INTO ATTENDANCE_PRESENT (NAME, DATE, TIME) VALUES (%s, CURDATE(), CURTIME())",
            (name,)
        )
        conn.commit()

    conn.close()

students = fetch_students()
known_faces = encode_faces(students)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(list(known_faces.values()), face_encoding)
        
        if True in matches:
            first_match_index = matches.index(True)
            name = list(known_faces.keys())[first_match_index]
            
            # Mark attendance only once per day
            mark_attendance(name)
        else:
            name = "Unknown"

        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left*4, top*4), (right*4, bottom*4), (0, 0, 255), 2)
        if name in list(known_faces.keys()):
         cv2.putText(frame, f'Attendence marked for {name} on {datetime.datetime.now().strftime("%Y-%m-%d")}', (left*4, top*4 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        else:
         cv2.putText(frame, name, (left*4, top*4 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        
    cv2.imshow("Face Recognition Attendance", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
import mysql.connector

def mark_absentees():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Password@123",
        database="AttendanceDB"
    )
    cursor = conn.cursor()

    # Fetch all students from Total_students
    cursor.execute("SELECT NAME FROM STUDENTS_RECORD")
    all_students = {row[0] for row in cursor.fetchall()}  # Convert to a set
    print(all_students)

    # Fetch students marked present today
    cursor.execute("SELECT NAME FROM ATTENDANCE_PRESENT WHERE DATE = CURDATE()")
    present_students = {row[0] for row in cursor.fetchall()}  # Convert to a set

    # Find absentees by subtracting sets
    absent_students = all_students - present_students

    # Insert absentees into ATTENDANCE_ABSENT table
    for student in absent_students:
        cursor.execute("INSERT INTO ATTENDANCE_ABSENT (NAME, DATE) VALUES (%s, CURDATE())", (student,))

    conn.commit()
    conn.close()
    print(f"Absentees recorded: {absent_students}")
mark_absentees()

{'sidhi', 'sandra', 'jordon', 'madhuri', 'Akash', 'Anushka'}
Absentees recorded: {'sandra', 'madhuri', 'jordon', 'Anushka'}
